In [1]:
import numpy as np
import pickle
import pandas as pd
from os.path import join
import warnings
import json
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from scipy import stats
import os

import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials

CURRENT_DIR = os.getcwd()
os.chdir(CURRENT_DIR)


## Script to train and validate the gradient boosting models

Replace x with the name of the directory where the train, test sets and indices are stored

Load training and test data

In [ ]:
data_train = pd.read_pickle(join("partitions_x", "train_df.pkl"))
data_test = pd.read_pickle(join("partitions_x", "test_df.pkl"))

len(data_train), len(data_test)

(12595, 2849)

Check for data leakage between train and test sets

In [5]:
test_sequence = set(data_test["sequence"])
train_sequence = set(data_train["sequence"])

common_sequence = test_sequence.intersection(train_sequence)
print(common_sequence)

set()


Load indices

In [ ]:
train_indices = list(pd.read_pickle(join("partitions_x", "CV_train_indices.pkl")))
test_indices = list(pd.read_pickle(join("partitions_x", "CV_test_indices.pkl")))

In [7]:
max_train_index = max([max(ind) for ind in train_indices])
max_test_index = max([max(ind) for ind in test_indices])

print("Max train index:", max_train_index)
print("Max test index:", max_test_index)

Max train index: 12594
Max test index: 12594


Check for data leakage between cv sets

In [8]:
cv_dfs = []

for i, (train_idx, test_idx) in enumerate(zip(train_indices, test_indices), 1):
    cv_train_df = data_train.iloc[train_idx].copy()
    cv_test_df = data_train.iloc[test_idx].copy()
    cv_dfs.append((cv_train_df, cv_test_df))
    
    globals()[f"cv_train_df{i}"] = cv_train_df
    globals()[f"cv_test_df{i}"] = cv_test_df
cv_dfs

[(         ECNumber EnzymeType                          Organism  \
  1        3.1.1.81   wildtype  Alicyclobacillus acidoterrestris   
  10334     5.4.4.6   wildtype    Colletotrichum gloeosporioides   
  6        3.2.1.20   wildtype     Caldanaerobacter subterraneus   
  11        6.3.2.1   wildtype                   Lotus japonicus   
  16       2.7.1.21   wildtype               Thermotoga maritima   
  ...           ...        ...                               ...   
  12568  1.13.11.24   wildtype                 Bacillus subtilis   
  12572   4.2.1.105   wildtype              Glycyrrhiza echinata   
  12576     5.3.1.4   wildtype            Bacillus licheniformis   
  12577   2.7.11.11   wildtype                      Homo sapiens   
  12594    2.8.2.21   wildtype                      Homo sapiens   
  
                                          Canonical SMILES  \
  1                               CCCCCCCCCC(=O)NC1CCOC1=O   
  10334                      CCCCCC=CCC=CC(CCCCCCC(=O)O)O

In [9]:
for i, (cv_train_df, cv_test_df) in enumerate(cv_dfs, 1):
    train_seq = set(cv_train_df["sequence"])
    test_seq = set(cv_test_df["sequence"])
    
    common_seq = train_seq.intersection(test_seq)

    if common_seq:
        print(f"Fold {i}: {len(common_seq)} common sequences")
    else:
        print(f"No common sequences found across round {i} cv train/test dfs")

No common sequences found across round 1 cv train/test dfs
No common sequences found across round 2 cv train/test dfs
No common sequences found across round 3 cv train/test dfs
No common sequences found across round 4 cv train/test dfs
No common sequences found across round 5 cv train/test dfs


In [10]:
for i, (train_idx, test_idx) in enumerate(zip(train_indices, test_indices), 1):
    max_train_idx = max(train_idx) if train_idx else -1
    max_test_idx = max(test_idx) if test_idx else -1
    print(f"Fold {i}: Max train index = {max_train_idx}, Max test index = {max_test_idx}")

Fold 1: Max train index = 12594, Max test index = 12592
Fold 2: Max train index = 12594, Max test index = 12593
Fold 3: Max train index = 12594, Max test index = 12587
Fold 4: Max train index = 12594, Max test index = 12589
Fold 5: Max train index = 12593, Max test index = 12594


### Training a model with concatenated esm2 enzyme and ChemBERTa2 substrate representations

Create input matrices

In [11]:
train_esm2 = np.array(list(data_train["esm2_ChemBERTa2"]))
train_X = train_esm2
train_Y = np.array(list(data_train["log_km"]))

test_esm2 = np.array(list(data_test["esm2_ChemBERTa2"]))
test_X = test_esm2
test_Y = np.array(list(data_test["log_km"]))

Hyperparameter optimization

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 100)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=tpe.suggest, max_evals = 100, trials=trials)

Training and validating model

In [ ]:
param = best 

with open(join("/gpfs/project/dit55hov/brenda_sabio/best_param.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=4)

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("training_results", "Pearson_CV_xgboost_bs_esm2_ChemBERTa2.npy"), np.array(Pearson))
np.save(join("training_results", "MSE_CV_xgboost_bs_esm2_ChemBERTa2.npy"), np.array(MSE))
np.save(join("training_results", "R2_CV_xgboost_bs_esm2_ChemBERTa2.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("training_results", "y_test_pred_xgboost_bs_esm2_ChemBERTa2.npy"), bst.predict(dtest))
np.save(join("training_results", "y_test_true_xgboost_bs_esm2_ChemBERTa2.npy"), test_Y)